In [2]:
import pandas as pd
from datasets import load_from_disk, Dataset
import os

# 1. 데이터셋 경로 설정
dataset_path = "/home/jovyan/CHJ/Mol-LLM_Custom/dataset/train/GSAI-ML-LLaDA-8B-Instruct_string+graph_q32_test_3.3M_0415_verified_filtered_512"
save_path = dataset_path + "_sampled_100"

def sample_dataset_fast_pandas(path, n_samples=100, task_col="Task"):
    print(f"Loading dataset from: {path}")
    
    try:
        ds = load_from_disk(path)
    except FileNotFoundError:
        print("Error: 해당 경로에서 데이터셋을 찾을 수 없습니다.")
        return None

    # 컬럼 이름 확인
    if task_col not in ds.column_names:
        if "task" in ds.column_names:
            task_col = "task"
            print(f"Warning: '{task_col}' 컬럼을 대신 사용합니다.")
        else:
            print(f"Error: 데이터셋에 '{task_col}' 컬럼이 없습니다.")
            return None

    print(f"Original Dataset Size: {len(ds):,}")
    
    # ---------------------------------------------------------
    # [핵심] Pandas로 변환하여 고속 처리
    # ---------------------------------------------------------
    print("Converting to Pandas DataFrame... (This might take a moment)")
    df = ds.to_pandas()
    
    print(f"Sampling {n_samples} items per '{task_col}'...")
    
    # 1. 전체 데이터를 무작위로 섞습니다 (random_state로 재현성 확보)
    #    이렇게 하면 뒤에서 그냥 위에서부터 잘라도(head) 랜덤 샘플링과 같은 효과입니다.
    df_shuffled = df.sample(frac=1, random_state=42)
    
    # 2. Task별로 그룹화한 뒤, 상위 n_samples개만 남깁니다.
    #    데이터가 n_samples보다 적으면 알아서 전부 가져오고, 많으면 n개만 자릅니다.
    #    for loop 없이 내부적으로 최적화된 연산입니다.
    sampled_df = df_shuffled.groupby(task_col).head(n_samples)
    
    print(f"Sampled DataFrame Size: {len(sampled_df):,}")
    
    # ---------------------------------------------------------
    # 다시 Hugging Face Dataset으로 변환
    # ---------------------------------------------------------
    final_ds = Dataset.from_pandas(sampled_df)
    
    # to_pandas() -> from_pandas() 과정에서 생기는 인덱스 컬럼 제거
    if "__index_level_0__" in final_ds.column_names:
        final_ds = final_ds.remove_columns(["__index_level_0__"])
        
    print(f"Final Dataset Size: {len(final_ds):,}")
    
    return final_ds

# 실행
if __name__ == "__main__":
    sampled_dataset = sample_dataset_fast_pandas(dataset_path, n_samples=100)

    if sampled_dataset:
        print(f"Saving sampled dataset to: {save_path}")
        sampled_dataset.save_to_disk(save_path)
        print("Done.")

/home/jovyan/miniconda3/envs/MolDA_CHJ/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading dataset from: /home/jovyan/CHJ/Mol-LLM_Custom/dataset/train/GSAI-ML-LLaDA-8B-Instruct_string+graph_q32_test_3.3M_0415_verified_filtered_512
Original Dataset Size: 70,565
Converting to Pandas DataFrame... (This might take a moment)
Sampling 100 items per 'task'...
Sampled DataFrame Size: 1,900
Final Dataset Size: 1,900
Saving sampled dataset to: /home/jovyan/CHJ/Mol-LLM_Custom/dataset/train/GSAI-ML-LLaDA-8B-Instruct_string+graph_q32_test_3.3M_0415_verified_filtered_512_sampled_100


Saving the dataset (1/1 shards): 100%|██████████| 1900/1900 [00:00<00:00, 73347.91 examples/s]

Done.
